)clear completely

In [1]:
)cd /home/hemmecke/g/qeta/tmp
)set mess type off
)r qetalibs
)r etamacros.input )quiet

# Problem formulation

Let $p(n)$ denote the number of partitions of the natural number $n$. Show that for every natural number $k$ the expression $p(11k+6)$ is divisible by 11.

The solution is described in the article (http://www.sciencedirect.com/science/article/pii/S0747717117300147):

    @Article{Hemmecke:DancingSambaRamanujan:2018,
      author =       {Ralf Hemmecke},
      title =        {Dancing Samba with {R}amanujan Partition
                      Congruences},
      journal =      {Journal of Symbolic Computation,
      year =         2018,
      JournalUrl =   {http://www.sciencedirect.com/science/journal/07477171},
      URL =          {http://www.sciencedirect.com/science/article/pii/S0747717117300147},
      DOI =          {10.1016/j.jsc.2017.02.001},
      volume =       {84},
      pages =        {14--24},
      ISSN =         {0747-7171},
      keywords =     {Partition identities, Number theoretic algorithm,
                      Subalgebra basis},
      abstract =     {The article presents an algorithm to compute a
                      $C[t]$-module basis $G$ for a given subalgebra $A$
                      over a polynomial ring $R=C[x]$ with a Euclidean
                      domain $C$ as the domain of coefficients and $t$ a
                      given element of $A$. The reduction modulo $G$
                      allows a subalgebra membership test. The algorithm
                      also works for more general rings $R$, in particular
                      for a ring $R\subset C((q))$ with the property that
                      $f\in R$ is zero if and only if the order of $f$ is
                      positive. As an application, we algorithmically
                      derive an explicit identity (in terms of quotients
                      of Dedekind $\eta$-functions and Klein's
                      $j$-invariant) that shows that $p(11n+6)$ is
                      divisible by 11 for every natural number $n$ where
                      $p(n)$ denotes the number of partitions of $n$.}
     }

in particlular, what we here call `algebraBasis` corresponds to the `samba` algorithm
from the article.

The article is also available as <a href="http://www.risc.jku.at/publications/download/risc_5338/DancingSambaRamanujan.pdf">RISC report 16-06</a>.

## Explanation of the session

Let X be an algebra that can be used in with the algorithm `samba`, but which has a polynomial attached to record the operations that have been done with the original element. This somehow simulates a similar idea as attaching a unit matrix to a matrix and then computing the row echelon form of the matrix. The unit matrix turns into the transformation matrix.

Since we want to show divisibility by 11 of $p(11k+6)$ for every $k$ where $p$ is the partition function, the main computation is done with a coefficient ring $C=\mathbb{Z}_{(11)}$ being integers localized at 11, i.e. rational numbers that have no denominator divisible by 11.

In [2]:
C ==> IntegerLocalizedAtPrime 11
X ==> QEtaExtendedAlgebra(C, L, Pol);
xembed(a,b)==>embed(a,b)
et ==> eta$ETAZ -- et(n) encodes \eta(n \tau) as a powerseries in q=2i\pi\tau.

The following function simply maps the integer coefficients of the series into the localized coefficient domain or into the rational number domain Q.

In [3]:
abmap(A, B, x) ==> map((c: A): B +-> c::B, x)$Finite0SeriesFunctions2(A, B, 'q, 'q, 0, 0)
zcmap(x) ==> abmap(Z, C, x)
cqmap(x) ==> abmap(C, Q, x)
zqmap(x) ==> abmap(Z, Q, x)

# Relations in terms of the $M_i$'s

In [4]:
level := 22;
rgens := xetaQuotientMonoidExponentVectors level;
divs := divisors(level)$IntegerNumberTheoryFunctions;
etaquotgensZ: List EtaQuotient(Z, LZ) := [etaQuotient(divs, r) for r in rgens];
for x in etaquotgensZ repeat if not zero? prefactor x then error "nonzero prefactor";
eqgensZ: List LZ := [retract x for x in etaquotgensZ];
eqgens: List L := [zcmap x for x in eqgensZ];
[qetaGrade x for x in eqgens]

   [5, 5, 6, 5, 7, 8, 9]


The generators of the $\eta$-quotients of level 22 are given by these exponents for the $\eta$ functions (with arguments being the divisors of 22).
The variable `rgens` corresponds to the generators of $R^\infty(22)$.

In [5]:
rgens

   [[7, - 3, 3, - 7], [- 4, 8, 4, - 8], [- 2, 6, 6, - 10], [- 1, 1, 11, - 11],
    [0, 4, 8, - 12], [2, 2, 10, - 14], [4, 0, 12, - 16]]


In [6]:
msyms := indexedSymbols("M", #eqgens)

   [M1, M2, M3, M4, M5, M6, M7]


In [7]:
xeqgens: List X := [xembed(x, s) for x in eqgens for s in msyms]

      - 5      - 4       - 3       - 2      - 1           1
   [[q    - 7 q    + 17 q    - 14 q    + 2 q    - 21 + O(q ), M1],
      - 5      - 4      - 3      - 2       - 1           1
    [q    + 4 q    + 6 q    + 8 q    + 13 q    + 12 + O(q ), M2],
      - 6      - 5    - 4      - 3    - 2      - 1      0
    [q    + 2 q    - q    - 2 q    - q    - 6 q    + O(q ), M3],
      - 5    - 4    - 3      - 2      - 1          1
    [q    + q    + q    + 2 q    + 2 q    + 3 + O(q ), M4],
      - 7      - 5      - 3      - 1
    [q    - 4 q    + 2 q    + O(q   ), M5],
      - 8      - 7      - 6      - 5      - 4      - 2
    [q    - 2 q    - 3 q    + 6 q    + 2 q    + O(q   ), M6],
      - 9      - 8      - 7      - 6      - 5      - 4      - 3
    [q    - 4 q    + 2 q    + 8 q    - 5 q    - 4 q    + O(q   ), M7]]


We take a minimal (in terms the order in q) element from the generators and make it special.
Then we compute an algebra basis for $C[M_1,\ldots,M_7]$.

In [8]:
tex: X := xeqgens.1

     - 5      - 4       - 3       - 2      - 1           1
   [q    - 7 q    + 17 q    - 14 q    + 2 q    - 21 + O(q ), M1]


In [9]:
abex := algebraBasis(tex, xeqgens)$QEtaAlgebraBasis(C, X, QEtaExtendedComputation(C, L, X))

   [
          - 3       - 1     2    1                   2      3
     [11 q    + 11 q    + 11 (- --) + 11 2 q + 11 2 q  + O(q ),
                                32

          1        5        21           1           213       85        1    2
         --- M7 - --- M6 + --- M5 + (- ---- M1 + 11 ----)M4 - --- M3 + ---- M1
         128      128      128         1024         1024      128      1024
       + 
          505
         ---- M1
         1024
       ]
     ,
      - 4      - 3      - 2                2   1      1
    [q    - 2 q    + 2 q    + 3 - 4 q + O(q ), - M4 - - M1],
                                               8      8
      - 6      - 5    - 4      - 3    - 2      - 1      0
    [q    + 2 q    - q    - 2 q    - q    - 6 q    + O(q ), M3],
      - 7      - 5      - 3      - 1
    [q    - 4 q    + 2 q    + O(q   ), M5],
      - 8      - 7      - 6      - 5      - 4      - 2
    [q    - 2 q    - 3 q    + 6 q    + 2 q    + O(q   ), M6]]


In [10]:
#abex

   5


In [11]:
[second x for x in abex]

   [
        1        5        21           1           213       85        1    2
       --- M7 - --- M6 + --- M5 + (- ---- M1 + 11 ----)M4 - --- M3 + ---- M1
       128      128      128         1024         1024      128      1024
     + 
        505
       ---- M1
       1024
     ,
    1      1
    - M4 - - M1, M3, M5, M6]
    8      8


## Relation for divisibility of $p(11k+6)$ by 11

Let's first define $F$ as on top of page 30 of \cite{Radu:RamanujanKolberg:2015}, i.e.
$$
F(\tau) = q^{\frac{13}{24}}\frac{\eta(\tau)^{10}\eta(2\tau)^2\eta(11\tau)^{11}}{\eta(22\tau)^{22}} \sum_{k=0}^{\infty}p(11k+6)q^k
$$

First we define the partion series.

In [12]:
ps: LZ := partitionSeries(1)$QFunctions(Z, LZ)

              2      3      4      5      6
   1 + q + 2 q  + 3 q  + 5 q  + 7 q  + O(q )


Then we select the sub-series given by the (11n+6)-th term.

In [13]:
r11: LZ := choose((n: Z): Z +-> 11*n+6, ps, 0)

                      2          3           4            5      6
   11 + 297 q + 3718 q  + 31185 q  + 204226 q  + 1121505 q  + O(q )


In [14]:
pfZ := retract(q24(13)$EtaQuotient(Z, LZ)*etaQuotient(divs, [10,2,11,-22])$EtaQuotient(Z, LZ));

In [15]:
q: LZ := 'q :: LZ

   q


In [16]:
fe := pfZ * r11;

In [17]:
fex: X := xembed(zcmap fe, F)

   [
           - 14          - 13           - 12           - 11     2    - 10
       11 q     + 11 17 q     + 11 101 q     + 11 336 q     + 11 84 q
     + 
                - 9      - 8
       11 2026 q    + O(q   )
     ,
    F]


Now we can reduce this element `fex` by the respective algebra basis, taking into account that `tex` is our special $t$ that we used in the construction of the algebra basis of the $\eta$-quotients of level 22.

As can be seen from the sorted monomials of the representaion, $F$ can be expressed as $F=11p(M_1,\ldots, M_7)$ where $p$ is a polynomial having coefficient denominators not divisible by 11.

In [18]:
rex := reduce(fex, tex, abex)

       12
   [O(q  ),

         2 767         2   3         2 1059           57        2 1493
       11  --- M7 + (11  ---- M1 + 11  ----)M6 + (11 --- M1 + 11  ----)M5
           340           1360           340          272           340
     + 
             1     2     2  4497        3  789          1867        2 619
       (11 ----- M1  + 11  ----- M1 + 11  ----)M4 + (11 ---- M1 + 11  ---)M3
           10880           10880          2720          1360          340
     + 
               1     3          1011   2     2   1647        1        3
       11 (- -----)M1  + 11 (- -----)M1  + 11 (- ----)M1 - ---- F + 11 (- 1)
             10880             10880             2720      1360
     ]


In [19]:
zero? rex

   true


In [20]:
mrex := [(inv(unitPart leadingCoefficient x)::C)*x for x in monomials second rex]

      2      2         2                2         2      2         3
   [11 M7, 11 M1 M6, 11 M6, 11 M1 M5, 11 M5, 11 M1 M4, 11 M1 M4, 11 M4,
                2         3       2    2         3
    11 M1 M3, 11 M3, 11 M1 , 11 M1 , 11 M1, F, 11 ]


In [21]:
smaller11(x:Pol,y:Pol):Boolean == exponent leadingCoefficient x < exponent leadingCoefficient y

In [22]:
smallerT(x:Pol,y:Pol):Boolean == degree(x, 'T) < degree(y, 'T)

In [23]:
sort(smaller11, mrex)

            2       3                 2                2      2      2
   [F, 11 M1 , 11 M1 , 11 M1 M3, 11 M1 M4, 11 M1 M5, 11 M1, 11 M3, 11 M1 M4,
      2      2         2      2      3    3
    11 M5, 11 M1 M6, 11 M6, 11 M7, 11 , 11 M4]


Let's normalize the leading coefficient of $F$ to 1.

In [24]:
rexq := second rex;

In [25]:
cfxq := coefficient(rexq, 'F, 1)

       1
   - ----
     1360


In [26]:
lcfxq := leadingCoefficient cfxq

       1
   - ----
     1360


In [27]:
rxq := inv(lcfxq)::C*rexq

       2                2            2
     11 (- 3068)M7 + (11 (- 3)M1 + 11 (- 4236))M6
   + 
                       2
     (11 (- 285)M1 + 11 (- 5972))M5
   + 
            1   2     2   4497        3   789
     (11 (- -)M1  + 11 (- ----)M1 + 11 (- ---))M4
            8               8              2
   + 
                        2                 1   3      1011   2     2 1647
     (11 (- 1867)M1 + 11 (- 2476))M3 + 11 - M1  + 11 ---- M1  + 11  ---- M1 + F
                                          8            8              2
   + 
       3
     11 1360


Let's plug in the original $\eta$-quotients into `rxq`.

In [28]:
ez := eval(rxq, c+->c::L, cons('F,msyms), cons(first fex,eqgens))$PolynomialEvaluation(C, L)

      - 4
   O(q   )


In [29]:
zero? ez

   true


# Express in Klein's J-invariant

Let
$$U(v):=t^v q \prod_{n=1}^{\infty}(1-q^{11n})\sum_{k=0}^{\infty}p(11k+6)q^k.$$

Below, we show that $U(7)=11 t^2 h(t, j, j_p)$ where $j$, $j_p$, and $t$ are Laurent series with integer coefficients and $h$ is a polynomial with rational coefficients having no denominator that is divisible by 11. In other words, we can give a relation $z U(7) = 11 t^2 k(t, j, j_p)$ with an integer polynomial $k$ and an integer $z$ such that $\gcd(z,11)=1$.

We choose Klein's $j$-invariant (https://en.wikipedia.org/wiki/J-invariant#The_q-expansion_and_moonshine) expressed as a Laurent series in terms of $q = \exp(2\pi i \tau)$ as the series $j$ above.

The series $j_p$ is given by replacing $q$ by $q^{11}$ in $j$.

The series $t$ is given by
$$t = \left(\frac{\eta(\tau)}{\eta(11\tau)}\right)^{12}$$
(again in terms of $q = \exp(2\pi i \tau)$) where $\eta$ is Dedekind's $\eta$-function (https://en.wikipedia.org/wiki/Dedekind_eta_function#Definition).


## Construction of Klein's $j$-invariant

<font color='red'>TODO: Need reference for the following representation of $j$.</font>

https://mathoverflow.net/questions/71704/computing-the-q-series-of-the-j-invariant

See Serre: A course in Arithmetic

http://functions.wolfram.com/EllipticFunctions/KleinInvariantJ/27/02/03/

Klein's $j$-invariant can be given by the expression $$j(\tau) = \frac{(\xi(\tau)+256)^3}{\xi(\tau)^2}$$ where $$\xi(\tau) = \left(\frac{\eta(\tau)}{\eta(2\tau)}\right)^{24}.$$ 
In the code below we do not mention the $\tau$ explicitly. The resulting series are immediately expressed in terms of $q = \exp(2\pi i\tau)$.

In [30]:
xi(n:P): LZ == retract((et(n)/et(2*n))^24);
iota(n: P): LZ == (xi(n)+256)^3/xi(n)^2;
iz: LZ := iota 1

    - 1                              2              3                4      5
   q    + 744 + 196884 q + 21493760 q  + 864299970 q  + 20245856256 q  + O(q )


Klein's $j$-invariant can be given by the expression $$j(\tau) = E_4^3/\Delta$$ where
$$\Delta=\frac{E_4^3-E_6^2}{1728}=\eta(\tau)^{24}$$
and the normalize Eisenstein series $E_4$ and $E_6$ are given by
$$E_4=1+240 \sum_{n=1}^\infty \sigma_3(n)q^n$$ and
$$E_6=1-504 \sum_{n=1}^\infty \sigma_5(n)q^n$$ where $q = \exp(2\pi i\tau)$ and
$\sigma_k(n)$ denotes the sum of the $k$-th power of the positive divisors of $n$.

All the above are series with integers coefficients.
For our computation, however, we compute with integers localized at the prime 11, i.e. these are rational numbers with denominators not divisible by 11.

The following function simply maps the integer coefficients of the series into the localized coefficient domain and eventually creates an object that attaches the indeterminate $J$ to the series.

In [31]:
i: L := kleinJInvariant()$QFunctions(C, L);
j: X := xembed(i, J); first j, second j

     - 1                              2              3                4      5
   [q    + 744 + 196884 q + 21493760 q  + 864299970 q  + 20245856256 q  + O(q ),
    J]


In [32]:
ip: L := multiplyExponents(i, 11);
jp: X := xembed(ip, Jp); first jp, second jp

     - 11      - 5
   [q     + O(q   ), Jp]


### The series $t$

The value $t$ is given by $$t = \left(\frac{\eta(\tau)}{\eta(11\tau)}\right)^{12}$$
in terms of $q = \exp(2\pi i \tau)$.

In [33]:
tz: LZ := retract(etaQuotient([1,11],[12,-12])$ETAZ);
tt: L := zcmap tz;
t: X := xembed(tt, T); first t, second t

     - 5       - 4       - 3            - 2            - 1            1
   [q    - 12 q    + 54 q    + 11 (- 8)q    + 11 (- 9)q    + 540 + O(q ), T]


## Construct elements suitable for the algebraBasis algorithm

In [34]:
m1: X := t;

We need some auxiliary values $m_2$ and $m_3$.

In [35]:
m2: X := j*t^3; first m2, second m2

   [
        - 16        - 15           - 14               - 13              - 12
       q     + 708 q     + 170694 q     + 11 1349272 q     + 203137101 q
     + 
                    - 11      - 10
       11 69897060 q     + O(q    )
     ,
       3
    J T ]


In [36]:
m3: X := jp*t; first m3, second m3

   [
        - 16       - 15       - 14            - 13            - 12        - 11
       q     - 12 q     + 54 q     + 11 (- 8)q     + 11 (- 9)q     + 540 q
     + 
          - 10
       O(q    )
     ,
    Jp T]


We want yet another value to end up in an algebra basis of orders -2 and -3.

We ignore this for now, because with the data that produces a relation among $F$, $J$, $J_p$, and $T$, the computation did not finish.

In [37]:
m := jp; mp := j;
while (d := qetaGrade(m)$X) > 0 repeat (c := qetaLeadingCoefficient(m)$X; e:N := d::N; m := m-c*j^e; mp:=mp-c*jp^e)

In [38]:
mp

       - 121      - 115
   [- q      + O(q     ),

           11            10                   9                   8
       - Jp   + 11 744 Jp   + 11 (- 2570796)Jp  + 11 4880620256 Jp
     + 
                             7     2                  6
       11 (- 5550817150590)Jp  + 11 351821430873072 Jp
     + 
         2                        5     2                       4
       11 (- 147929969197380488)Jp  + 11 35519316022933293120 Jp
     + 
         2                            3     2                           2
       11 (- 4373015221856525734395)Jp  + 11 224874476512864841450600 Jp
     + 
         2
       11 (- 3096214928567780111700972)Jp + J
     ]


In [39]:
mgens := [m1,m2,m3]

      - 5       - 4       - 3            - 2            - 1            1
   [[q    - 12 q    + 54 q    + 11 (- 8)q    + 11 (- 9)q    + 540 + O(q ), T],

     [
          - 16        - 15           - 14               - 13              - 12
         q     + 708 q     + 170694 q     + 11 1349272 q     + 203137101 q
       + 
                      - 11      - 10
         11 69897060 q     + O(q    )
       ,
         3
      J T ]
     ,

     [
          - 16       - 15       - 14            - 13            - 12        - 11
         q     - 12 q     + 54 q     + 11 (- 8)q     + 11 (- 9)q     + 540 q
       + 
            - 10
         O(q    )
       ,
      Jp T]
     ]


### The algebra basis computation

Then we compute the algebra basis for the elements $t=m_1$, $m_2$, and $m_3$.

Since we compute in an extended algebra. The second component of each entry in the basis gives its expression in terms of the original generators.

In [41]:
--approx. 75 seconds
ab: List X := algebraBasis(t, mgens)$QEtaAlgebraBasis(C, X, QEtaExtendedComputation(C, L, X)); #ab

   12


Show the powers of 11 in the coefficients of the representation (in terms of J, Jp, T) of the algebra basis elements.

In [42]:
[[exponent(x) for x in coefficients second ab.n] for n in 2..#ab]

   [
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
      1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
      1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 0, 0,
      1, 1, 1, 0, 3, 3, 0, 0, 0, 3, 1, 3, 4, 4, 4, 5, 4, 6, 6, 6, 8, 8]
     ,

     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 3, 2, 0, 1, 0, 1,
      0, 2, 2, 3, 0, 1, 0, 1, 0, 3, 3, 1, 2, 3, 1, 3, 4, 1, 1, 1, 4, 4, 2, 2,
      4, 4]
     ,

     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
      1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0,
      0, 0, 0, 2, 2, 0, 0, 2, 0, 2, 3, 0, 0, 0, 3, 4, 3, 4, 3]
     ,

     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

The (absolute value of the) order and the power of 11 in the leading coefficient of the elements in the algebra basis.

In [43]:
[[qetaGrade i, unitCanonical qetaLeadingCoefficient i] for i in ab]

          18         8         4          3          3          3
   [[7, 11  ], [8, 11 ], [9, 11 ], [11, 11 ], [12, 11 ], [13, 11 ], [14, 1],
                    2
    [16, 1], [17, 11 ], [18, 11], [11 2, 1], [23, 1]]


We define
$$U(v):=t^v q \prod_{n=1}^{\infty}(1-q^{11n})\sum_{k=0}^{\infty}p(11k+6)q^k.$$

Let's do this step by step. We take the partition series from above.

In [44]:
ps

              2      3      4      5      6
   1 + q + 2 q  + 3 q  + 5 q  + 7 q  + O(q )


We also take the sub-series given by the (11n+6)-th term of ps as defined earlier. The coefficients of the series `r11` we want to check for divisibility by 11. 

In [45]:
r11

                      2          3           4            5      6
   11 + 297 q + 3718 q  + 31185 q  + 204226 q  + 1121505 q  + O(q )


Putting all together (except the factor $t^v$) gives the following expression.

In [46]:
fz: LZ := q*eulerFunction(11)$QFunctions(Z, LZ)*r11;
f: X := xembed(zcmap fz, F)

   [
                     2           3            4             5              6
       11 q + 11 27 q  + 11 338 q  + 11 2835 q  + 11 18566 q  + 11 101955 q
     + 
          7
       O(q )
     ,
    F]


Now let's check what the smallest $v>0$ is such that $U(v)=t^v f$ is in the $\mathbb{Q}[t]$-module generated by $\{1, z_1, \ldots, z_{12}\}$ where the $z_i$ are given by the list `ab`.

Although for the coefficient domain $\mathbb{Q}$ we find $v=3$, with the coefficients being in $C=\mathbb{Z}_{(11)}$, the reduction to zero only starts to work with $v=7$.

In [47]:
qab := [cqmap first x for x in ab];
[zero? reduce(cqmap(first(t^v * f)), cqmap first t, qab)$QEtaReduction(Q, LQ) for v in 1..7]

   [false, false, true, true, true, true, true]


With $v=3$ the reduction does not lead to 0 in case the coefficients are non-rational, but rather lie in $\mathbb{Z}_{(11)}$.

In [48]:
reduce(t^3*f, t, ab)

   [
           - 13      3285  - 12      8841  - 11         7246  - 10
       11 q     + 11 ---- q     + 11 ---- q     + 11 (- ----)q
                      247             247                247
     + 
             174366  - 9         441459  - 8      - 7
       11 (- ------)q    + 11 (- ------)q    + O(q   )
               247                 247
     ,
            1         3              1     3             1
    (11 -------- J - --- F + 11 (- -----))T  + 11 (- --------)Jp T]
        16183440     247           22477             16183440


In [49]:
[zero? reduce(t^v * f, t, ab) for v in 1..10]

   [false, false, false, false, false, false, true, true, true, true]


Since all involved series in the algebra basis `ab` have their representation in terms of $J$, $Jp$, and $T$ attached, we have found a relation in $F$, $J$, $Jp$, and $T$.

In [50]:
result := reduce(t^7 * f, t, ab);
zero? result
res := second result;
variables res

   true
   [T, Jp, J, F]


The leading coefficient is a rational number whose numerator and denominator are not divisible by 11, i.e., the `exponent` (of 11) of the coefficient is zero.

In [51]:
cf := coefficient(res, 'F, 1);
lcf := leadingCoefficient cf
exponent lcf

   -
       50529448729713161372287812465113088257245028539357223404943346605701491_
        0196350406914325450897464531108991411920230123849040077382874946968087_
        5868834233890730496196195678258203428161507327770190169677428557801518_
        8852077633178670384003886841425286833330748013108959170157009521929939_
        2364587220349275723435521887053129596441634027981689079125493090431957_
        3739780050639020602561970984140995364820467459976808958043119593302527_
        4162950721734165386853604324141839348908851647037357776192759698205848_
        6327434321491044515367122675166500106736654520882272455721927098830141_
        0333112394124991910339662961807518758534534739462800787773095703807932_
        6335925721959170197688698398943757445863890728712310642160059439018866_
        3314450678836261041424060095577812916933836358608449552816430495240474_
        0218071460226565108923655342664000348847772188242322953619344490741683_
        337158306823435970898550780

Let's normalize the coefficient in front of the variable $F$ such that it becomes 1, i.e., we multiply by the inverse of the coefficient.

In [52]:
r := (inv(unitPart lcf)::C)*res;

Let us investigate the coefficients of all monomials of $r$.

In [53]:
mr := [(inv(unitPart leadingCoefficient x)::C)*x for x in monomials r];
smaller11(x:Pol,y:Pol):Boolean == exponent leadingCoefficient x < exponent leadingCoefficient y
smallerT(x:Pol,y:Pol):Boolean == degree(x, 'T) < degree(y, 'T)
sort(smaller11, mr)
sort(smallerT, mr)

       7         6      2 6       2 6           6       3 6      2 7        7
   [F T , 11 Jp T , 11 J T , 11 Jp T , 11 J Jp T , 11 Jp T , 11 J T , 11 J T ,
             7         7       2 7        8      8         8      2 8
    11 J Jp T , 11 Jp T , 11 Jp T , 11 J T , 11 T , 11 Jp T , 11 J T ,
             8       2 8         2 8       3 8      9      2 9         9
    11 J Jp T , 11 Jp T , 11 J Jp T , 11 Jp T , 11 T , 11 J T , 11 Jp T ,
        2    9           9       2 9         2 9       3 9        10      10
    11 J Jp T , 11 J Jp T , 11 Jp T , 11 J Jp T , 11 Jp T , 11 J T  , 11 T  ,
        2 10         10      2    10           10       2 10      11      2 11
    11 J T  , 11 Jp T  , 11 J Jp T  , 11 J Jp T  , 11 Jp T  , 11 T  , 11 J T  ,
        3 11           11         11      2    11       2 11         2 11
    11 J T  , 11 J Jp T  , 11 Jp T  , 11 J Jp T  , 11 Jp T  , 11 J Jp T  ,
        12      2 12        12      3 12         12      2    12           12
    11 T  , 11

The above shows that we have found a relation
$$U(7):=t^7 q \prod_{n=1}^{\infty}(1-q^{11n})\sum_{k=0}^{\infty}p(11k+6)q^k = 11 t^2 h(t,j,j_p)$$
where $h$ is a polynomial in $\mathbb{Z}_{(11)}[t,j,j_p]$.

Let us create output that can directly pasted into a journal paper.

In [54]:
tups := [(d:=degree(x, [T,J,Jp]); [d.1-2,d.2,d.3]) for x in rest sort(smaller11, mr)];
#tups
for x in sort tups repeat print hconcat(x::Tuple(Z)::OutputForm, ","::Symbol)

   161
   (0, 0, 0),
   (0, 0, 1),
   (0, 0, 2),
   (1, 0, 0),
   (1, 0, 1),
   (1, 0, 2),
   (1, 0, 3),
   (2, 0, 0),
   (2, 0, 1),
   (2, 0, 2),
   (2, 0, 3),
   (2, 0, 4),
   (2, 1, 0),
   (2, 1, 1),
   (3, 0, 0),
   (3, 0, 1),
   (3, 0, 2),
   (3, 0, 3),
   (3, 0, 4),
   (3, 1, 0),
   (3, 1, 1),
   (3, 1, 2),
   (4, 0, 0),
   (4, 0, 1),
   (4, 0, 2),
   (4, 0, 3),
   (4, 0, 4),
   (4, 1, 0),
   (4, 1, 1),
   (4, 1, 2),
   (4, 1, 3),
   (4, 2, 0),
   (5, 0, 0),
   (5, 0, 1),
   (5, 0, 2),
   (5, 0, 3),
   (5, 0, 4),
   (5, 1, 0),
   (5, 1, 1),
   (5, 1, 2),
   (5, 1, 3),
   (5, 2, 0),
   (5, 2, 1),
   (6, 0, 0),
   (6, 0, 1),
   (6, 0, 2),
   (6, 0, 3),
   (6, 0, 4),
   (6, 1, 0),
   (6, 1, 1),
   (6, 1, 2),
   (6, 1, 3),
   (6, 2, 0),
   (6, 2, 1),
   (6, 2, 2),
   (7, 0, 0),
   (7, 0, 1),
   (7, 0, 2),
   (7, 0, 3),
   (7, 0, 4),
   (7, 1, 0),
   (7, 1, 1),
   (7, 1, 2),
   (7, 1, 3),
   (7, 2, 0),
   (7, 2, 1),
   (7, 2, 2),
   (7, 3, 0),
   (8, 0, 0),
   (8, 0, 1),
   (8, 0, 2),

Size of numerator and denominators of the coefficients of the linear combination of the powerproducts.

In [55]:
sort [floor((log(abs(numer(x::Q)::Float))/log(10.0))::Float) for x in coefficients r]
sort [floor((log(abs(denom(x::Q)::Float))/log(10.0))::Float) for x in coefficients r]

   [0.0, 2324.0, 2324.0, 2325.0, 2325.0, 2325.0, 2325.0, 2326.0, 2326.0,
    2326.0, 2326.0, 2326.0, 2326.0, 2327.0, 2327.0, 2327.0, 2329.0, 2332.0,
    2335.0, 2335.0, 2336.0, 2336.0, 2336.0, 2336.0, 2337.0, 2337.0, 2337.0,
    2337.0, 2338.0, 2338.0, 2339.0, 2340.0, 2342.0, 2343.0, 2343.0, 2344.0,
    2344.0, 2345.0, 2345.0, 2345.0, 2345.0, 2347.0, 2347.0, 2347.0, 2347.0,
    2347.0, 2347.0, 2349.0, 2349.0, 2350.0, 2350.0, 2351.0, 2351.0, 2351.0,
    2352.0, 2352.0, 2353.0, 2354.0, 2354.0, 2354.0, 2354.0, 2355.0, 2355.0,
    2356.0, 2356.0, 2356.0, 2356.0, 2356.0, 2357.0, 2357.0, 2357.0, 2357.0,
    2358.0, 2359.0, 2359.0, 2359.0, 2359.0, 2360.0, 2360.0, 2360.0, 2361.0,
    2361.0, 2361.0, 2362.0, 2362.0, 2362.0, 2363.0, 2364.0, 2364.0, 2364.0,
    2364.0, 2364.0, 2364.0, 2364.0, 2365.0, 2365.0, 2365.0, 2365.0, 2366.0,
    2366.0, 2366.0, 2366.0, 2367.0, 2367.0, 2367.0, 2367.0, 2368.0, 2369.0,
    2369.0, 2370.0, 2370.0, 2370.0, 2370.0, 2371.0, 2371.0, 2372.0, 2373.0,
    2373.0, 237

In [56]:
sort [degree(x, T) for x in mr]

   [2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6,
    6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8,
    8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10,
    10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11,
    11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13,
    13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14,
    14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16,
    16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18]


## Check the relation

Plugging in the series $f$, $j$, $j_p$, $t$, for the indeterminates $F$, $J$, $J_p$, $T$ in $r$ should result in the zero series.

In [57]:
-- approx. 45 seconds
fjjpt: List Symbol := [F, J, Jp, T];
z := eval(r, c+->c::L, fjjpt, [first f, first j, first jp, first t])$PolynomialEvaluation(C, L)
zero? z
z

      - 83
   O(q    )
   true
      12
   O(q  )


Let's check whether the relation interpreted with rational coefficients also yields zero. This checks against bugs in `IntegerLocalizedAtPrime`.

In [58]:
-- approx. 30 seconds
rq := r::Polynomial(Fraction Integer);
zq := eval(rq, c+->c::LQ, fjjpt, [cqmap first f, cqmap i, cqmap ip, zqmap tz])$PolynomialEvaluation(Q, LQ)
zero? zq
zq

      - 83
   O(q    )
   true
      12
   O(q  )


# The same algorithm can be used over the rational numbers $\mathbb{Q}$

However, that computation leads to only 5 elements in the algebra basis.

In [59]:
XQ ==> QEtaExtendedAlgebra(Q, LQ, PolQ);
jq:  XQ := xembed(cqmap i,  J);
jpq: XQ := xembed(cqmap ip, Jp);
tq:  XQ := xembed(zqmap tz,  T);
mqgens: List XQ := [tq, jq*tq^3, jpq*tq];
abq: List XQ := algebraBasis(tq, mqgens)$QEtaAlgebraBasis(Q, XQ, QEtaExtendedComputation(Q, LQ, XQ)); #abq

   4


In [60]:
[[qetaGrade i, unitCanonical qetaLeadingCoefficient i] for i in abq]

   [[7, 1], [8, 1], [9, 1], [11, 1]]


In [61]:
fq: XQ := xembed(zqmap fz, F)

                2         3          4           5            6      7
   [11 q + 297 q  + 3718 q  + 31185 q  + 204226 q  + 1121505 q  + O(q ), F]


As we have seen above, $U(3)$ lies in the $\mathbb{Q}[t]$-module generated by 1 and the elements of the algebra basis.

In [62]:
[zero? reduce(tq^v * fq, tq, abq)$QEtaReduction(Q, XQ) for v in 1..7]

   [false, false, true, true, true, true, true]


In [63]:
resultq := reduce(tq^3*fq, tq, abq);
resq := second resultq;
cfq := coefficient(resq, 'F, 1);
lcfq := leadingCoefficient cfq
rq := inv(lcfq)*resq;

   -
       12521111727577241016552261232773825912405848219540858433182030934460333_
        79433146400428686578188183211
     /
       30115817805964225077677784999144493849261483637326937531926638818488554_
        29918948961031907889283682210836


Reduction with respect to the rational basis is not useful to show divisibility of $fq$ by 11, since it leads to denominators of some coefficients of monomials that do not involve the variable $F$.

In [64]:
[unitCanonical(denom(x)::C)*((inv(leadingCoefficient(x)))*x)::Pol for x in monomials rq]

      40 2 17    40   17    40 17    40 2 16    40   16    40 16    40      15
   [11  J T  , 11  J T  , 11  T  , 11  J T  , 11  J T  , 11  T  , 11  J Jp T  ,
      40    15    40 2 15    40   15    40 15    40      14    40    14
    11  Jp T  , 11  J T  , 11  J T  , 11  T  , 11  J Jp T  , 11  Jp T  ,
      40 2 14    39   14    39 14    40  2 13    39      13    40    13
    11  J T  , 11  J T  , 11  T  , 11  Jp T  , 11  J Jp T  , 11  Jp T  ,
      39 2 13    39   13    39 13    40  2 12    39      12    40    12
    11  J T  , 11  J T  , 11  T  , 11  Jp T  , 11  J Jp T  , 11  Jp T  ,
      39 2 12    38   12    38 12    40  2 11    37      11    40    11
    11  J T  , 11  J T  , 11  T  , 11  Jp T  , 11  J Jp T  , 11  Jp T  ,
      38 2 11    38   11    38 11    38  2 10    38      10    36    10
    11  J T  , 11  J T  , 11  T  , 11  Jp T  , 11  J Jp T  , 11  Jp T  ,
      38 2 10    37   10    35 10    35  2 9    35      9    34    9    35 2 9
    11  J T  , 11  J T  , 11  T  , 11

# The relation in $M_i$'s with integer coefficients

Here $\mathbb{Z}$ is considered as a Euclidean domain wrt the absolute value as the Euclidean size function.

In [65]:
XZ ==> QEtaExtendedAlgebra(Z, LZ, PolZ)

In [66]:
xeqgensZ: List XZ := [xembed(x, s) for x in eqgensZ for s in msyms]

      - 5      - 4       - 3       - 2      - 1           1
   [[q    - 7 q    + 17 q    - 14 q    + 2 q    - 21 + O(q ), M1],
      - 5      - 4      - 3      - 2       - 1           1
    [q    + 4 q    + 6 q    + 8 q    + 13 q    + 12 + O(q ), M2],
      - 6      - 5    - 4      - 3    - 2      - 1      0
    [q    + 2 q    - q    - 2 q    - q    - 6 q    + O(q ), M3],
      - 5    - 4    - 3      - 2      - 1          1
    [q    + q    + q    + 2 q    + 2 q    + 3 + O(q ), M4],
      - 7      - 5      - 3      - 1
    [q    - 4 q    + 2 q    + O(q   ), M5],
      - 8      - 7      - 6      - 5      - 4      - 2
    [q    - 2 q    - 3 q    + 6 q    + 2 q    + O(q   ), M6],
      - 9      - 8      - 7      - 6      - 5      - 4      - 3
    [q    - 4 q    + 2 q    + 8 q    - 5 q    - 4 q    + O(q   ), M7]]


In [67]:
ttez := xeqgensZ.1
abez := algebraBasis(tez, xeqgensZ)$QEtaAlgebraBasis(Z, XZ, QEtaExtendedComputation(Z, LZ, XZ))
#abez

     - 5      - 4       - 3       - 2      - 1           1
   [q    - 7 q    + 17 q    - 14 q    + 2 q    - 21 + O(q ), M1]
         - 3       - 1                2      3
   [[88 q    + 88 q    + 176 q + 176 q  + O(q ), - 11 M4 + 8 M2 + 3 M1],
      - 4       - 3      - 2       - 1                 2
    [q    + 31 q    + 2 q    + 33 q    + 3 + 62 q + O(q ), - 4 M4 + 3 M2 + M1],
      - 6      - 5    - 4      - 3    - 2      - 1      0
    [q    + 2 q    - q    - 2 q    - q    - 6 q    + O(q ), M3],
      - 7      - 5      - 3      - 1
    [q    - 4 q    + 2 q    + O(q   ), M5],
      - 8      - 7      - 6      - 5      - 4      - 2
    [q    - 2 q    - 3 q    + 6 q    + 2 q    + O(q   ), M6]]
   5


In [68]:
fez: XZ := xembed(fe, F)

   [
           - 14        - 13         - 12         - 11          - 10          - 9
       11 q     + 187 q     + 1111 q     + 3696 q     + 10164 q     + 22286 q
     + 
          - 8
       O(q   )
     ,
    F]


Interestingly, we have to multiply by 4, because otherwise the reduction would not return 0.

In [69]:
rez := reduce(4*fez, tez, abez)
zero? rez
rz := second rez;
mrez := monomials rz;
[[factor leadingCoefficient x, degree x] for x in mrez]

       12
   [O(q  ),

       139799528 M6 + (- 19800 M1 - 718078856)M5
     + 
              2
       (176 M1  + 18031904 M1 - 16752297419)M4 + (- 51656 M1 + 3007344648)M3
     + 
                2                                      3             2
       (- 132 M1  - 13523928 M1 + 4586709248)M2 - 44 M1  - 4903712 M1
     + 
       - 593090333 M1 + 4 F - 18978271136
     ]
   true
      3  2                   3 2 2                3  2
   [[2 11 139 1039, M6], [- 2 3 5 11, M5 M1], [- 2 11 19 39043, M5],
      4         2     5  2                   3
    [2 11, M4 M1 ], [2 11 4657, M4 M1], [- 11 13 968173, M4],
        3                  3      2                  2           2
    [- 2 11 587, M3 M1], [2 3 7 11 239 619, M3], [- 2 3 11, M2 M1 ],
        3    2                8  2                 2      3
    [- 2 3 11 4657, M2 M1], [2 11 148073, M2], [- 2 11, M1 ],
        5            2        2                  2          5  3
    [- 2 11 13931, M1 ], [- 11 163 30071, M1], [2 , F], [

In [70]:
ez := eval(rz, c+->c::LZ, cons('F,msyms), cons(first fez,eqgensZ))$PolynomialEvaluation(Z, LZ)
zero? ez

      - 4
   O(q   )
   true


# Paule's proof via computation in $C[t,F]$

In [71]:
fep: X := xembed(zcmap(tz*fz), Y)

        - 4          - 3          - 2           - 1                          2
   [11 q    + 11 15 q    + 11 68 q    + 11 149 q    + 11 323 + 11 376 q + O(q ),
    Y]


In [72]:
t

     - 5       - 4       - 3            - 2            - 1            1
   [q    - 12 q    + 54 q    + 11 (- 8)q    + 11 (- 9)q    + 540 + O(q ), T]


In [73]:
abp := algebraBasis(t, [fep])$QEtaAlgebraBasis(C, X, QEtaExtendedComputation(C, L, X));
#abp
[second x for x in abp]

   4
        2   3   4
   [Y, Y , Y , Y ]


In [74]:
rep := reduce((fep)^5, t, abp)

       12
   [O(q  ),

        5     4      4      4             8    3
       Y  + 11 (- 5)Y  + (11 (- 251)T + 11 10)Y
     + 
          3         2     8             12        2
       (11 (- 4093)T  + 11 (- 930)T + 11  (- 10))Y
     + 
          4        3     7       2     12             16        5      4
       (11 (- 123)T  + 11 15468 T  + 11  (- 170)T + 11  5)Y + 11 (- 1)T
     + 
         7         3     11         2     16         20
       11 (- 1442)T  + 11  (- 1090)T  + 11  20 T + 11  (- 1)
     ]


In [75]:
-(second rep)::PolQ::PolZ::Polynomial(Factored(Z))

        5       4 4      4              8  3
     - Y  + 5 11 Y  + (11 251 T - 2 5 11 )Y
   + 
        3      2           8             12  2
     (11 4093 T  + 2 3 5 11 31 T + 2 5 11  )Y
   + 
          4    3    2    7      2         12           16       5 4
     (3 11 41 T  - 2 3 11 1289 T  + 2 5 11  17 T - 5 11  )Y + 11 T
   + 
           7     3         11     2    2    16      20
     2 7 11 103 T  + 2 5 11  109 T  - 2 5 11  T + 11


In [76]:
(factor(11^5*T^4+2*7*11^7*103*T^3+2*5*11^11*109*T^2-2^2*5*11^16*T+11^20))::Factored(Polynomial(Factored Z))

     5       4   3     2      2         7      11
   11 (T + 11 )(T  + 11 1321 T  - 3 7 11 T + 11  )


The above polynomial agrees with the relation that Peter Paule has on his Slide 32 of hia talk at "The 2016 International Number Theory Conference in honor of Krishna Alladi's 60th birthday" (Gainesville, March 17-21, 2016).

And now here with keeping the original tagging for $f$, i.e., in the next result, $Y=T y$.

In [77]:
abp := algebraBasis(t, [t*xembed(first f, 'y)])$QEtaAlgebraBasis(C, X, QEtaExtendedComputation(C, L, X));
#abp
[second x for x in abp]

   4
          2 2   3 3   4 4
   [T y, T y , T y , T y ]


In [78]:
rep := reduce(t^5*(f)^5, t, abp)

       12
   [O(q  ),

         4      4 4      4        4     8    3  3
       11 (- 5)T y  + (11 (- 251)T  + 11 10 T )y
     + 
          3         4     8        3     12       2  2
       (11 (- 4093)T  + 11 (- 930)T  + 11  (- 10)T )y
     + 
          4        4     7       3     12        2     16         5 5
       (11 (- 123)T  + 11 15468 T  + 11  (- 170)T  + 11  5 T)y + F T
     + 
         5      4     7         3     11         2     16         20
       11 (- 1)T  + 11 (- 1442)T  + 11  (- 1090)T  + 11  20 T + 11  (- 1)
     ]


In [79]:
-(second rep)::PolQ::PolZ::Polynomial(Factored(Z))

         4 4 4      4     4         8 3  3
     5 11 T y  + (11 251 T  - 2 5 11 T )y
   + 
        3      4           8    3         12 2  2
     (11 4093 T  + 2 3 5 11 31 T  + 2 5 11  T )y
   + 
          4    4    2    7      3         12    2       16       5 5     5 4
     (3 11 41 T  - 2 3 11 1289 T  + 2 5 11  17 T  - 5 11  T)y - F T  + 11 T
   + 
           7     3         11     2    2    16      20
     2 7 11 103 T  + 2 5 11  109 T  - 2 5 11  T + 11


# Checking with PowerSamba

In [64]:
QMEVS==>QEtaQuotientMonoidExponentVectorsStar
eqmevs==>etaQuotientMonoidExponentVectors$QMEVS
eqmevx==>etaQuotientMonoidExponentVectorsX$QMEVS
lc ==> leadingCoefficient

S ==> Symbol
EZ ==> Expression Z
expr x ==> x :: S :: EZ
PL ==> PolynomialCategoryLifting(N, SingletonAsOrderedSet, C, PZ, PQ)
LX ==> UnivariateLaurentSeries(CX, xsym, 0)
EQX ==> EtaQuotientX(Q, mx, CX, xi, LX)
EQXA ==> EtaQuotientExpansionAlgebra(CX, LX, level)

PZ ==> SparseUnivariatePolynomial Z
PQ ==> SparseUnivariatePolynomial Q
Rec ==> Record(root: P, elem: PZ)

unityroots(m: P): List P == (_
  divs: List P := [qcoerce(delta)@P for delta in divisors m];_
  rs := eqmevx m;_
  l: List P := empty(); _
  for r in rs repeat (_
    e := etaQuotient(m, divs, r)$SymbolicEtaQuotient;_
    l := cons(rootOfUnity e, l) _
  );_
  l_
)

In [65]:
m := level
mx: P := lcm unityroots(m)
xsym: Symbol := "x"::Symbol

   22
   2
   x


In [66]:
if mx > 2 then (_
    pz: PZ := cyclotomic(mx)$CyclotomicPolynomialPackage;_
    pq: PQ := map(n+->monomial(1$Q,1$N)$PQ, c+->c::Q::PQ, pz)$PL;_
    CX := SimpleAlgebraicExtension(Q, PQ, pq);_
    xi := generator()$CX_
) else (_
    CX := Q;_
    xi := -1_
)

   - 1


In [67]:
)clear prop divs
divs: List P := [qcoerce(d)@P for d in divisors m]

   [1, 2, 11, 22]


In [68]:
rs := eqmevx m

   [[- 7, 3, - 3, 7], [- 3, 7, - 7, 3], [3, - 7, 7, - 3], [7, - 3, 3, - 7],
    [11, - 11, - 1, 1], [- 11, 11, 1, - 1]]


In [70]:
lse := [etaQuotient(m, divs, r)$ SymbolicEtaQuotient for r in rs];
le := [etaQuotient(e)$EQX for e in lse];
-- [etaQuotient(divs, r)$ETA for r in rs]
eas := [etaQuotient(r, expansions e)$EQXA for e in le for r in rs]

   [
     [
           1   - 5     7   - 4    17   - 3     7   - 2     1   - 1    21
         ---- x    - ---- x    + ---- x    - ---- x    + ---- x    - ----
         3872        3872        3872        1936        1936        3872
       + 
          9       13   2    13   3    3   4     5   5      6
         --- x + ---- x  - ---- x  - --- x  + ---- x  + O(x )
         968     3872      1936      484      1936
       ,

            1     3       2   2    1   3    2   4    4   5    3   6    2   7
         - --- - --- x - --- x  - --- x  - --- x  + --- x  + --- x  - --- x
           121   121     121      121      121      121      121      121
       + 
          9   8    5   9    1   10      11
         --- x  + --- x  - --- x   + O(x  )
         121      121      121
       ,

            1    3      7  2   1  3   35  4   35  5   133  6   245  7   441  8
         - -- + -- x - -- x  + - x  - -- x  + -- x  - --- x  + --- x  - --- x
           32   32     32      2      32      16   

In [71]:
grds := [qetaGrades  ea for ea in eas]

   [[5, 0, 0, - 5], [0, - 5, 5, 0], [0, 5, - 5, 0], [- 5, 0, 0, 5],
    [- 5, 5, 0, 0], [5, - 5, 0, 0]]


In [203]:
rgens := xetaQuotientMonoidExponentVectors level

   [[7, - 3, 3, - 7], [- 4, 8, 4, - 8], [- 2, 6, 6, - 10], [- 1, 1, 11, - 11],
    [0, 4, 8, - 12], [2, 2, 10, - 14], [4, 0, 12, - 16]]
                                                    Type: List(List(Integer))


In [211]:
Mlse := [etaQuotient(m, divs, r)$ SymbolicEtaQuotient for r in rgens];
Mle := [etaQuotient(e)$EQX for e in Mlse];
-- [etaQuotient(divs, r)$ETA for r in rs]
M := [etaQuotient(r, expansions e)$EQXA for e in Mle for r in rgens]

                                              Type: List(SymbolicEtaQuotient)
Type: List(EtaQuotientX(Fraction(Integer),2,Fraction(Integer),-1,UnivariateLaurentSeries(Fraction(Integer),x,0)))
   [
     [
          121   - 10    121   - 9    121  - 8    605   - 7    1815  - 6
         ----- x     - ----- x    + ---- x    - ----- x    + ----- x
         32768         32768        8192        32768        32768
       + 
            2299  - 5    5445  - 4    7139  - 3   15125  - 2   19965  - 1   9559
         - ----- x    + ----- x    - ----- x    + ----- x    - ----- x    + ----
           32768        32768        32768        32768        32768        8192
       + 
            1
         O(x )
       ,

                15        16         17         18          19          20
         - 121 x   - 605 x   - 2299 x   - 7260 x   - 20449 x   - 52756 x
       + 
                   21           22           23            24            25
         - 127655 x   - 292699 x   - 642510 x   - 13

In [212]:
grds := [qetaGrades  ea for ea in M]

   [[10, - 15, 5, 0]]
                                                    Type: List(List(Integer))


In [75]:
mat:=matrix [[1, 1, 1, 1, 0, 0, 0, 0], [1, 2, 11, 22, 24, 0, 0, 0], [22, 11, 2, 1, 0, 24, 0, 0], [0, 1, 0, 1, 0, 0, 2, 0], [0, 0, 1, 1, 0, 0, 0, 2]]

   +1   1   1   1   0   0   0  0+
   |                            |
   |1   2   11  22  24  0   0  0|
   |                            |
   |22  11  2   1   0   24  0  0|
   |                            |
   |0   1   0   1   0   0   2  0|
   |                            |
   +0   0   1   1   0   0   0  2+


In [145]:
nmat := transpose matrix [members x for x in nullSpace mat];

                                                        Type: Matrix(Integer)


In [144]:
v := transpose matrix [[- 2, 6, 6, - 10,6,-1,2,2]];

                                                        Type: Matrix(Integer)


In [163]:
ch:=completeHermite(nmat)

              +1  3  0+          +0  0  0  2  0   5    5    8 +
              |       |          |                            |
              |0  5  0|          |0  0  0  0  0   0    1    0 |
              |       |          |                            |
              |0  0  1|          |0  0  0  0  1  - 1  - 1  - 2|
              |       |          |                            |
              |0  0  0|          |1  0  0  4  0  12    9   18 |
   [Hermite = |       |, eqMat = |                            |]
              |0  0  0|          |0  1  0  1  0   0    2    0 |
              |       |          |                            |
              |0  0  0|          |0  0  1  1  0   0    0    2 |
              |       |          |                            |
              |0  0  0|          |0  0  0  5  0  12   11   20 |
              |       |          |                            |
              +0  0  0+          +0  0  0  0  2  - 2  - 2  - 5+
                Type: Record(Hermite: M

In [173]:
transpose(lv := ch.eqMat * v)

   [1  2  1  0  0  0  0  0]
                                                        Type: Matrix(Integer)


In [191]:
lmat:Matrix(Z):=transpose matrix [[-1,2,5]]

   +- 1+
   |   |
   | 2 |
   |   |
   + 5 +
                                                        Type: Matrix(Integer)


In [192]:
transpose(ch.Hermite*lmat)

   [5  10  5  0  0  0  0  0]
                                                        Type: Matrix(Integer)


In [188]:
5*transpose lv

   [5  10  5  0  0  0  0  0]
                                                        Type: Matrix(Integer)


In [197]:
1/5*nmat*lmat

   +- 2 +
   |    |
   | 6  |
   |    |
   | 6  |
   |    |
   |- 10|
   |    |
   | 6  |
   |    |
   |- 1 |
   |    |
   | 2  |
   |    |
   + 2  +
                                              Type: Matrix(Fraction(Integer))


In [198]:
mat

   +1   1   1   1   0   0   0  0+
   |                            |
   |1   2   11  22  24  0   0  0|
   |                            |
   |22  11  2   1   0   24  0  0|
   |                            |
   |0   1   0   1   0   0   2  0|
   |                            |
   +0   0   1   1   0   0   0  2+
                                             Type: Matrix(NonNegativeInteger)


In [201]:
rgens := [[11, -1, -11, 1], [24, 0, -24, 0], [-36, -4, 40, 0]]

   [[11, - 1, - 11, 1], [24, 0, - 24, 0], [- 36, - 4, 40, 0]]
                                                    Type: List(List(Integer))


In [204]:
rs

   [[- 7, 3, - 3, 7], [- 3, 7, - 7, 3], [3, - 7, 7, - 3], [7, - 3, 3, - 7],
    [11, - 11, - 1, 1], [- 11, 11, 1, - 1]]
                                                    Type: List(List(Integer))


In [206]:
imat:Matrix Z := mat

   +1   1   1   1   0   0   0  0+
   |                            |
   |1   2   11  22  24  0   0  0|
   |                            |
   |22  11  2   1   0   24  0  0|
   |                            |
   |0   1   0   1   0   0   2  0|
   |                            |
   +0   0   1   1   0   0   0  2+
                                                        Type: Matrix(Integer)


In [213]:
rgens := [[-25,29,-5,1],[-12,16,-4,0],[-36,40,-4,0]]

   [[- 25, 29, - 5, 1], [- 12, 16, - 4, 0], [- 36, 40, - 4, 0]]
                                                    Type: List(List(Integer))


In [214]:
matrix rgens

   +- 25  29  - 5  1+
   |                |
   |- 12  16  - 4  0|
   |                |
   +- 36  40  - 4  0+
                                                        Type: Matrix(Integer)


In [219]:
imat(1..5,1..4)* transpose matrix rgens

   +  0     0      0  +
   |                  |
   |  0    - 24    0  |
   |                  |
   |- 240  - 96  - 360|
   |                  |
   | 30     16    40  |
   |                  |
   + - 4   - 4    - 4 +
                                                        Type: Matrix(Integer)


In [1]:
1+1

   2
                                                        Type: PositiveInteger


In [34]:
)set mess type